<a href="https://colab.research.google.com/github/ValentinCord/HandOnAI_1/blob/main/Fire_Data_Cleaner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **I. Reading data**

In [1]:
!/opt/bin/nvidia-smi
!rm -rf sample_data

Tue Nov  1 08:39:09 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os

In [3]:
bases_path_after="bases"
if os.path.exists(bases_path_after) == False:
    os.makedirs(bases_path_after)

In [4]:
!wget https://cluster.ig.umons.ac.be/HackIA21/databases/FIRE_DATABASE_1.tar
!tar xf FIRE_DATABASE_1.tar -C 'bases' --one-top-level
!rm FIRE_DATABASE_1.tar

!wget https://cluster.ig.umons.ac.be/HackIA21/databases/FIRE_DATABASE_2.tar
!tar xf FIRE_DATABASE_2.tar -C 'bases' --one-top-level
!rm FIRE_DATABASE_2.tar

!wget https://cluster.ig.umons.ac.be/HackIA21/databases/FIRE_DATABASE_3.tar
!tar xf FIRE_DATABASE_3.tar -C 'bases' --one-top-level
!rm FIRE_DATABASE_3.tar

--2022-11-01 08:39:20--  https://cluster.ig.umons.ac.be/HackIA21/databases/FIRE_DATABASE_1.tar
Resolving cluster.ig.umons.ac.be (cluster.ig.umons.ac.be)... 193.190.209.220
Connecting to cluster.ig.umons.ac.be (cluster.ig.umons.ac.be)|193.190.209.220|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 466772992 (445M) [application/octet-stream]
Saving to: ‘FIRE_DATABASE_1.tar’

FIRE_DATABASE_1.tar 100%[===================>] 445.15M  65.4MB/s    in 6.8s    

2022-11-01 08:39:27 (65.0 MB/s) - ‘FIRE_DATABASE_1.tar’ saved [466772992/466772992]

--2022-11-01 08:39:28--  https://cluster.ig.umons.ac.be/HackIA21/databases/FIRE_DATABASE_2.tar
Resolving cluster.ig.umons.ac.be (cluster.ig.umons.ac.be)... 193.190.209.220
Connecting to cluster.ig.umons.ac.be (cluster.ig.umons.ac.be)|193.190.209.220|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1163223040 (1.1G) [application/octet-stream]
Saving to: ‘FIRE_DATABASE_2.tar’

FIRE_DATABASE_2.tar 100%[====

# **II. Cleaning data**
1. Create **db path**
2. **Merge** all fire database **without 'copie' files**
3. Find and delete **abberant** images
4. Find and delete **duplicate** images

In [7]:
!rm -r db
!mkdir -p db/fire db/start_fire db/no_fire

In [8]:
!find bases/FIRE_DATABASE_1/fire ! -name '*copie*' -exec cp "{}" db/fire \;
!find bases/FIRE_DATABASE_2/fire ! -name '*copie*' -exec cp "{}" db/fire \;
!find bases/FIRE_DATABASE_3/fire ! -name '*copie*' -exec cp "{}" db/fire \;

!find bases/FIRE_DATABASE_1/start_fire ! -name '*copie*' -exec cp "{}" db/start_fire \;
!find bases/FIRE_DATABASE_2/start_fire ! -name '*copie*' -exec cp "{}" db/start_fire \;
!find bases/FIRE_DATABASE_3/start_fire ! -name '*copie*' -exec cp "{}" db/start_fire \;

!find bases/FIRE_DATABASE_1/no_fire ! -name '*copie*' -exec cp "{}" db/no_fire \;
!find bases/FIRE_DATABASE_2/no_fire ! -name '*copie*' -exec cp "{}" db/no_fire \;
!find bases/FIRE_DATABASE_3/no_fire ! -name '*copie*' -exec cp "{}" db/no_fire \;

cp: -r not specified; omitting directory 'bases/FIRE_DATABASE_1/fire'
cp: -r not specified; omitting directory 'bases/FIRE_DATABASE_2/fire'
cp: -r not specified; omitting directory 'bases/FIRE_DATABASE_3/fire'
cp: -r not specified; omitting directory 'bases/FIRE_DATABASE_1/start_fire'
cp: -r not specified; omitting directory 'bases/FIRE_DATABASE_2/start_fire'
cp: -r not specified; omitting directory 'bases/FIRE_DATABASE_3/start_fire'
cp: -r not specified; omitting directory 'bases/FIRE_DATABASE_1/no_fire'
cp: -r not specified; omitting directory 'bases/FIRE_DATABASE_2/no_fire'
cp: -r not specified; omitting directory 'bases/FIRE_DATABASE_3/no_fire'


In [ ]:
img_aberrant = {
    'fire': ['00000335.jpg',
             'images - 2022-05-04T104033.252.jpeg',
             'images - 2022-05-04T104034.807.jpeg',
             'images - 2022-05-04T104044.688.jpeg',
             'images - 2022-05-04T104043.935.jpeg',
             'images - 2022-05-04T104047.349.jpeg'],
    'start_fire': [],
    'no_fire': []
    }

# **III. Adding data**
1. Add **sunset**, **fox**, **automn** images as no fire
2. Discuss about start fire and **fog similarity**

# **IV. Store database on google.drive**



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cp -av "/content/all_data/" "/content/gdrive/MyDrive/Challenge1/"

# **V. Delete duplicate images**